# Walkthrough Challenge 1 - Initialize the Kernel and Run Semantic Functions

Duration: 30 minutes

## Overview
- In this challenge, you will start by configuring and initializing the kernel.
- You will also learn how to use Semantic Kernel to define and run Semantic Functions with C#.
- In the end, you will use Semantic Kernel to dynamically generate prompts at runtime.

## Prerequisites

- Please ensure that you have completed the [Setup](../setup/setup.ipynb) before starting this challenge.

### Task 1: Configure and Initialize Semantic Kernel

⚠️ Note: You should have already completed all tasks on the [Setup](../setup/setup.ipynb). If you have not, please go back and complete it now.

#### Step 1: Load Semantic Kernel settings

In this step, we will load the Semantic Kernel settings that we created in the [Setup](../setup/setup.ipynb) notebook.

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.0.1"

#!import ../setup/config/Settings.cs

Installed Packages Microsoft.SemanticKernel, 1.0.1

#### Step 2: Initialize Semantic Kernel

In [15]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.TemplateEngine;
using Kernel = Microsoft.SemanticKernel.Kernel;
using Microsoft.DotNet.Interactive;
using InteractiveKernel = Microsoft.DotNet.Interactive.Kernel;

var builder = Kernel.CreateBuilder();

// Configure AI service credentials used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile("../setup/config/settings.json");

if (useAzureOpenAI)
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
else
    builder.AddOpenAIChatCompletion(model, apiKey, orgId);

var kernel = builder.Build();

#### Step 3: Create a Semantic Function

We will create a Semantic Function that will be used to generate prompts at runtime.

A prompt is a question, statement or a command that the user wants the model to respond to. In this case, we will create a prompt that will ask the model to summarize a given text input.

Notice that we are introducing a variable `{{$input}}`, which will be replaced with the user input at runtime.

In [3]:
string skPrompt = """
{{$input}}

Summarize the above content.
The summary should be two sentences long.
""";

If you want to inspect how the prompt is generated at runtime, you can use a `KernelPromptTemplateFactory` to do so. It will render the prompt, based on the template in the previous step.

In [4]:
var promptTemplateConfig = new PromptTemplateConfig(skPrompt);

var promptTemplateFactory = new KernelPromptTemplateFactory();
var promptTemplate = promptTemplateFactory.Create(promptTemplateConfig);

var renderedPrompt = await promptTemplate.RenderAsync(kernel);

Console.WriteLine(renderedPrompt);



Summarize the above content.
The summary should be 3 sentences long.


Along with the prompt, let's also configure the parameters for the Semantic Function execution.
`MaxTokens` is the maximum number of tokens that the model will generate. `Temperature` is a hyperparameter that controls the randomness of the model. `TopP` is another hyperparameter that controls the diversity of the model.

In [5]:
var executionSettings = new OpenAIPromptExecutionSettings
{
    MaxTokens = 500,
    Temperature = 0.2,
    TopP = 0.5
};

With the text prompt and the parameters ready, we can now create the Semantic Function, that the kernel can execute.

In [6]:
var summaryFunction = kernel.CreateFunctionFromPrompt(skPrompt, executionSettings);

#### Step 4: Run Semantic Function

Let's initialize some content for the summarization function. We are going to use a sample text from [Wikipedia](https://en.wikipedia.org/wiki/Hanging_Gardens_of_Babylon), which is about the Hanging Gardens of Babylon, one of the Seven Wonders of the Ancient World.

In [8]:
var input = """
The Hanging Gardens of Babylon were one of the Seven Wonders of the Ancient World listed by Hellenic culture.
They were described as a remarkable feat of engineering with an ascending series of tiered gardens containing a wide variety of trees, shrubs, and vines, resembling a large green mountain constructed of mud bricks.
It was said to have been built in the ancient city of Babylon, near present-day Hillah, Babil province, in Iraq.
The Hanging Gardens' name is derived from the Greek word κρεμαστός (kremastós, lit. 'overhanging'), which has a broader meaning than the modern English word "hanging" and refers to trees being planted on a raised structure such as a terrace.
""";

And we can now use the summary function to summarize the input text:

In [9]:
var summaryResult = await kernel.InvokeAsync(summaryFunction, new() { ["input"] = input });

Console.WriteLine(summaryResult);

The Hanging Gardens of Babylon were one of the Seven Wonders of the Ancient World, renowned for their elaborate design featuring tiered gardens with a diverse array of flora. They were an engineering marvel made of mud bricks, resembling a verdant mountain, and were located in the ancient city of Babylon, in what is now Iraq. The term "hanging" in their name comes from the Greek for "overhanging," referring to the gardens being set on terraces.


The code above goes through all the steps of creating a kernel, rendering a prompt, and executing the summarization function. However, we can simplify this process by using the `InvokePromptAsync` method, which will do all the steps above in one go:

In [10]:
string skPrompt = """
{{$input}}

Summarize the content above.
The summary should be two sentences long.
""";

var result = await kernel.InvokePromptAsync(skPrompt, new() { ["input"] = input });

Console.WriteLine(result);

The Hanging Gardens of Babylon were one of the Seven Wonders of the Ancient World, renowned for their terraced design filled with diverse vegetation. They were an extraordinary example of engineering, located in the ancient city of Babylon, which is near today's Hillah in Iraq.


Here is another example that will categorize the input text of an Hotel support ticket into one of the categories: `booking`, `cancellation`, `refund`, `complaint`, `other` and will also provide a priority for the ticket: `low`, `medium`, `high`.

Notice that in the example we are going to provide some examples for the model to learn from. This is called `few shot learning`, and it will improve the accuracy of the model response.

In [23]:
string skPrompt = """
{{$input}}

Categorize the content above in one of the following categories:
["booking", "cancellation", "refund", "complaint", "other"]

Attribute one of the following priorities to the above content:
["low", "medium", "high"]

The response should be a JSON object with the following structure:
{
    "category": "booking",
    "priority": "high"
}

Example 1:
USER: When I entered the room, I found that the bed was not made.
RESPONSE: {"category": "complaint", "priority": "high"}
""";

var hotelSupportTicket = await InteractiveKernel.GetInputAsync("Enter the hotel support ticket: ");

var result = await kernel.InvokePromptAsync(skPrompt, new() { ["input"] = hotelSupportTicket });

Console.WriteLine(result);

{
    "category": "other",
    "priority": "high"
}


Experiment with the example above and try to provide different inputs to see how the model responds. You can also try to provide different examples for the model to learn from.

You successfully completed challenge 1! 🚀🚀🚀